In [4]:
import pandas as pd
import numpy as np
import time

df = pd.read_csv('puzzle_batches/small_batch.csv')
df = df[['FEN','Moves','Themes']].dropna()
#df

In [5]:
# Execute computer move

import chess

#print(df['FEN'])
for row in df.itertuples():
    board = chess.Board(row.FEN)

    computer_move = row.Moves.split()[0]
    board.push_uci(computer_move)
    df.loc[row.Index].FEN = board.fen()

df.drop(['Moves'], inplace=True, axis=1)

In [6]:
# Create valid_themes columns

start = time.time()

used_themes = {
    "advancedPawn", "advantage", "bishopEndgame", "capturingDefender", "crushing", "equality", "kingsideAttack", "defensiveMove", "endgame",
    "exposedKing", "fork", "hangingPiece", "interference", "knightEndgame", "middlegame", "opening", "pawnEndgame", "pin", "promotion", "queenEndgame",
    "queenRookEndgame", "queensideAttack", "rookEndgame", "skewer", "trappedPiece", "underPromotion", "xRayAttack", "zugzwang",
}

df['valid_themes'] = ""
for row in df.itertuples():
    row_themes = set(row.Themes.split())
    print(row.Themes.split())
    print(row_themes)
    valid_themes = row_themes.intersection(used_themes)
    #print(f"valid themes {valid_themes}")

    df.loc[row.Index].valid_themes = " ".join(valid_themes)

themes_dummies = df.valid_themes.str.get_dummies(' ').add_prefix('theme_')
df = pd.concat([df, themes_dummies], axis=1)

end = time.time()
print(end - start)
#df.drop(['Themes', 'valid_themes'], inplace=True, axis=1)

SyntaxError: invalid syntax (<ipython-input-6-5799122eff88>, line 16)

In [23]:
# Create board positions with piece vectors

start = time.time()

squares = chess.SQUARE_NAMES

# pawn, knight, bishop, rook, king
# queen will be encoded as bishop + rook
pieces = ['p', 'n', 'b', 'r', 'k'] 
for s in squares:
    for p in pieces:
        df[f"{s}.{p}"] = 0

end = time.time()
print(end - start)

,endgame
pawnEndgame,0.159832
kingsideAttack,0.231171
crushing,0.183202
bishopEndgame,0.102757
mateIn2,0.069840
long,0.046296
clearance,0.015348
discoveredAttack,0.046663
queenEndgame,0.099836
fork,0.039698


In [104]:
# Transform FEN into board positions and pieces

start = time.time()

for row in df.itertuples():
    board = chess.Board(row.FEN)

    # color -> false = preto / true = branco
    for pos, piece in board.piece_map().items():
        square_name = chess.square_name(pos)
        df[f"{square_name}.{piece.lower()}"] = 1 if piece.color else -1


end = time.time()
print(end - start)

r2q3k/ppp3pB/3p3B/8/3b4/2N2P1b/PnQ2P2/R3R1K1 b - - 0 20
8/p2n2pk/3p1p1p/4pP1B/2PpP3/P2P1QRP/1r1q2PK/8 w - - 5 33
r1r3k1/p4ppp/4p3/8/3P4/4P3/qPR2PPP/2Q3K1 w - - 0 25
1r4k1/1q3p2/1p1P2p1/p1pQ3p/2P4P/1R4P1/5PK1/8 w - - 4 37
8/r3k2p/2p1p1p1/1rP1B3/p4R2/4P1PP/PP6/4K3 w - - 2 35
2k3r1/p7/2P5/3p1b1p/3B2Pq/1P2P2P/P6Q/2R2K2 b - - 1 31
6k1/6p1/1R4P1/4r3/2b5/8/P4K2/8 w - - 1 39
r3r1k1/5ppp/8/1q1p1Q2/p1p1n3/2P5/PP2RPPP/2B1R1K1 w - - 0 23
0.07243752479553223
